Import SysMLv2 Model:

In [1]:
import syside

EXAMPLE_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_FILE_PATH = EXAMPLE_DIR + "omg_single_example.sysml"
MODEL_FILE_PATH = EXAMPLE_DIR + "omg_introduction.sysml" # very complex model
(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Check Metamodel structure:

In [ ]:
for doc in model.user_docs:
    print(doc)
    with doc.lock() as locked:
        print(f"Model sexp:\n {syside.sexp(locked.root_node)}")

Check PartUsage:

In [ ]:
for PartUsage in model.nodes(syside.PartUsage):
    print(PartUsage)
    print(f"\tPartUsage::part_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tItemUsage::item_definitions: {len(list(PartUsage.part_definitions))}")
    print(f"\tOccurrenceUsage::individual_definition: {PartUsage.individual_definition}")
    print(f"\tElement::declared_name: {PartUsage.declared_name}")
    print(f"\tElement::documentation: {len(list(PartUsage.documentation))}")
    print(f"\tElement::element_id: {PartUsage.element_id}")
    print(f"\tElement::owner: {PartUsage.owner}")
    print(f"\tElement::owning_membership: {PartUsage.owning_membership}")
    print(f"\tElement::owning_namespace: {PartUsage.owning_namespace}")
None

Export to JSON:

In [5]:
from pathlib import Path

assert len(model.user_docs) == 1

with model.user_docs[0].lock() as locked:
    json_output = syside.json.dumps(
        locked.root_node, syside.SerializationOptions.minimal()
    )

# Use current working directory (instead of __file__)
# output_path = Path.cwd() / "omg_single_example.json"
output_path = Path.cwd() / "omg_introduction.json"

# Write JSON file
output_path.write_text(json_output, encoding="utf-8")

print(f"JSON saved to: {output_path}")


JSON saved to: /Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/omg_introduction.json


Create cypher script for neo4j:

In [12]:
import json

# with open("omg_single_example.json") as f:
with open("omg_introduction.json") as f:
    data = json.load(f)

cypher = []

# 1. Create all nodes
for e in data:
    props = []
    for key in ("@id", "@type", "declaredName", "memberName", "operator"):
        val = e.get(key)
        if val:
            field = "id" if key == "@id" else "type" if key == "@type" else key
            props.append(f'{field}:"{val}"')
    # cypher.append(f'CREATE (:`Node` {{{", ".join(props)}}});') # original
    # cypher.append(f'CREATE ({e.get("@type")}:Node {{{", ".join(props)}}});') # using @type as node label name
    cypher.append(f'CREATE (:{e.get("@type")} {{{", ".join(props)}}});') # using @type as node label

# 2. Create relationships
rels = {
    "owningRelatedElement": "OWNS",
    "ownedRelationship": "OWNS_REL",
    "memberElement": "MEMBER_OF",
    "ownedRelatedElement": "OWNS_RELATED",
    "owningRelationship": "OWNING_REL",
    "general": "GENERAL",
    "specific": "SPECIFIC",
    "type": "TYPE_OF",
    "typedFeature": "TYPED_FEATURE",
}
for e in data:
    src_id = e["@id"]
    for k, rel in rels.items():
        if k in e:
            targets = e[k] if isinstance(e[k], list) else [e[k]]
            for t in targets:
                tid = t.get("@id")
                if tid:
                    td_index = next((i for i, item in enumerate(data) if item["@id"] == t.get("@id")), None) # target data index
                    if td_index is None:
                        print("td_index is None!")
                    else:    
                        cypher.append(
                            # f'MATCH (a:Node {{id:"{src_id}"}}), (b:Node {{id:"{tid}"}}) ' # original
                            f'MATCH (a:{e.get("@type")} {{id:"{src_id}"}}), (b:{data[td_index]['@type']} {{id:"{tid}"}}) ' #
                            f'CREATE (a)-[:{rel}]->(b);'
                        )

with open("omg_introduction.cypher", "w") as f:
    f.write("\n".join(cypher))


td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
td_index is None!
